In [ ]:
from pyupbit.request_api import _call_public_api
import requests
import bs4
import pandas as pd
import time
import numpy as np
import telegram
import datetime


start_time = time.time()
pd.set_option('mode.chained_assignment',  None)

def tether_price():          # 크롤링으로 테더 가격 얻기 0.06~0.5초 소요
    upbit_url = "https://api.upbit.com/v1/ticker?markets=KRW-BTC"
    html = requests.get(upbit_url)
    upbit_price = bs4.BeautifulSoup(html.content, "html.parser").text.split('"trade_price":')[1].split(',"prev_')[0]
    binance_url = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT'
    html = requests.get(binance_url)
    binance_price = bs4.BeautifulSoup(html.content, "html.parser").text.split('"price":"')[1].split('"}')[0]
    usdt_price = float(upbit_price) / float(binance_price)
    return usdt_price


def upbit_price_get(dollar_won = 1300.0):          # 업비트 데이터 받기. 환율 정보 받는 거 빼고0.12초 소요
    def get_orderbook(tickers="KRW-BTC"):
        url = "https://api.upbit.com/v1/orderbook"
        exit_num = 1
        while exit_num:
            try:
                contents = _call_public_api(url, markets=tickers)[0]
                exit_num=0
            except:
                time.sleep(3)
                exit_num=1
                print('오류 발생',time.strftime('%c'))
        return contents

    def upbit_coin_list():
        url = "https://s3.ap-northeast-2.amazonaws.com/crix-production/crix_master"
        html = requests.get(url)
        bsobj = bs4.BeautifulSoup(html.content, "html.parser")
        bsobj_json = pd.read_json(bsobj.text)
        krwlisted_df = bsobj_json[bsobj_json['quoteCurrencyCode'] == 'KRW']
        krwlisted_df = krwlisted_df[krwlisted_df['delistingDate'].isna() == True]
        krwlisted_df = krwlisted_df[krwlisted_df['listingDate'].isna() == False]
        # 화폐가 krw인 것 중, delisting된 게 아닌 것 중, listing 된 적이 있는 것.
        krw_list = []
        for i in list(krwlisted_df['code']):
            krw_list.append(i[11:])

        return krw_list

    coin_list = upbit_coin_list()
    order_str = ''
    for i in coin_list:
        order_str += i + ','
    order_str = order_str[:-1]
    orderbook_list = get_orderbook(order_str)

    result_df = []
    for i in range(len(orderbook_list)):
        ticker = orderbook_list[i]['market'][4:]
        ask_price = orderbook_list[i]['orderbook_units'][0]['ask_price']
        bid_price = orderbook_list[i]['orderbook_units'][0]['bid_price']
        result_df.append({'exchange': 'upbit', 'ticker': ticker, 'buyable_price': ask_price/dollar_won, 'sellable_price': bid_price/dollar_won})

    return pd.DataFrame(result_df)


def binance_price_get():            # 바이낸스 데이터 받기. 0.1~0.13초 소요
    url = "https://api.binance.com/api/v3/ticker/bookTicker"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text)
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.contains('DOWN|UP|BULL|BEAR') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith(('USDT', 'BUSD'))]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='USDT', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='BUSD', repl='',regex=False)
    orderbook_list = orderbook_list[orderbook_list['bidQty'] != 0]  # 호가에 물량이 없는 거래 정지 종목 제외

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'binance'
    result_df['buyable_price'] = orderbook_list['askPrice']
    result_df['sellable_price'] = orderbook_list['bidPrice']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def poloniex_price_get(vol_condition = 50000):        # 폴로닉스 데이터 받기. 0.5~0.8초 소요. 데이터 받는 게 오래 걸림..
    url = "https://poloniex.com/public?command=returnTicker"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text)
    orderbook_list = bsobj_json.transpose().reset_index(drop=False, inplace=False)           # 인덱스 초기화 (이건 지우면 안 됨)
    orderbook_list.rename(columns={"index": "ticker"}, inplace=True)                         # 종목명의 컬럼 이름 바꿔줌
    orderbook_list = orderbook_list[orderbook_list['ticker'].str.contains('DOWN|UP|BULL|BEAR|TIME') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list = orderbook_list[orderbook_list['ticker'].str.startswith('USDT')]  # USDT로 시작하는 종목명만 남김
    orderbook_list['ticker'] = orderbook_list['ticker'].str.replace(pat='USDT_', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['baseVolume']>vol_condition]          # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['ticker']
    result_df['exchange'] = 'poloniex'
    result_df['buyable_price'] = orderbook_list['lowestAsk']
    result_df['sellable_price'] = orderbook_list['highestBid']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def huobi_global_price_get(vol_condition = 50000):          # 후오비 글로벌 데이터 받기. 약 0.25초 소요
    url = "https://api.huobi.pro/market/tickers"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"data":')[1].split(',"status"')[0])
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('usdt')]  # USDT로 끝나는 종목명만 남김
    orderbook_list = orderbook_list[orderbook_list['vol'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='usdt', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol'].str.upper()
    result_df['exchange'] = 'huobi_global'
    result_df['buyable_price'] = orderbook_list['ask']
    result_df['sellable_price'] = orderbook_list['bid']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def kucoin_price_get(vol_condition = 50000):
    url = "https://api.kucoin.com/api/v1/market/allTickers"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"ticker":')[1][:-2])
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 종목만 찾음
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.contains('3S|3L') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='-USDT', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['volValue'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'Kucoin'
    result_df['buyable_price'] = orderbook_list['sell']
    result_df['sellable_price'] = orderbook_list['buy']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def kraken_price_get(vol_condition = 50000):
    vol_condition = 50000
    url = "https://api.kraken.com/0/public/AssetPairs"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    ticker_list = pd.read_json(bsobj.text.split('"result":')[1][:-1])[:1].transpose()['altname']
    ticker_list = ticker_list[ticker_list.str.endswith(('USD', 'USDT'))]  # USD나 USDT로 끝나는 종목만 모음

    ticker_data = ''
    for i in ticker_list:
        ticker_data += i + ','

    ticker_data = ticker_data[:-1]
    url = "https://api.kraken.com/0/public/Ticker?pair=" + ticker_data
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"result":')[1][:-1]).transpose()

    result_list = []
    for i in range(len(orderbook_list)):
        ticker = orderbook_list.iloc[i].name
        buyable_price = float(orderbook_list.iloc[i]['a'][0])
        sellable_price = float(orderbook_list.iloc[i]['b'][0])
        volume = float(orderbook_list.iloc[i]['v'][1]) * float(sellable_price)
        if volume > vol_condition:
            result_list.append(['kraken', ticker, buyable_price, sellable_price])

    result_df = pd.DataFrame(result_list, columns=['exchange', 'ticker', 'buyable_price', 'sellable_price'])
    result_df['ticker'] = result_df['ticker'].str.split('USD').str.get(0)  # USD 기준으로 스플릿 후 앞의 것을 골라준다.
    return result_df


def gate_io_price_get(vol_condition = 50000):  # 0.3~0.4초 소요, close만 나옴..
    url = "https://api.gateio.ws/api/v4/spot/tickers"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text)
    orderbook_list = orderbook_list[orderbook_list['currency_pair'].str.endswith('USDT')]  # USD나 USDT로 끝나는 종목만 모음
    orderbook_list = orderbook_list[orderbook_list['currency_pair'].str.contains('3S|3L') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list['currency_pair'] = orderbook_list['currency_pair'].str.replace(pat='_USDT', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['quote_volume'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['currency_pair']
    result_df['exchange'] = 'gate.io'
    result_df['buyable_price'] = orderbook_list['lowest_ask'].astype('float')
    result_df['sellable_price'] = orderbook_list['highest_bid'].astype('float')
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def ftx_price_get(vol_condition = 50000):    # 0.05~0.1초 소요
    url = "https://ftx.com/api/markets"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"result":')[1][:-2])
    orderbook_list = orderbook_list[orderbook_list['name'].str.endswith('USD')]  # USDT로 끝나는 종목명만 남김
    orderbook_list = orderbook_list[orderbook_list['name'].str.contains('BULL|BEAR|HALF|HEDGE') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list = orderbook_list[orderbook_list['volumeUsd24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집
    orderbook_list['symbol'] = orderbook_list['name'].str.replace(pat='/USD', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol'].str.upper()
    result_df['exchange'] = 'FTX'
    result_df['buyable_price'] = orderbook_list['ask']
    result_df['sellable_price'] = orderbook_list['bid']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def binance_future_price_get():            # 바이낸스 선물 시장 데이터 받기. 0.1~0.13초 소요
    url = "https://fapi.binance.com/fapi/v1/ticker/bookTicker"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text)
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.contains('DOWN|UP|BULL|BEAR|DEFI') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='USDT', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['bidQty'] != 0]  # 호가에 물량이 없는 거래 정지 종목 제외

    index1000 = orderbook_list[orderbook_list['symbol'].str.contains('1000')]['bidPrice'].index
    orderbook_list.loc[index1000,'bidPrice'] = orderbook_list.loc[index1000,'bidPrice'] / 1000  # 1000 붙은 코인들 가격도 1000토막으로 맞춰줌
    orderbook_list.loc[index1000, 'askPrice'] = orderbook_list.loc[index1000, 'askPrice'] / 1000
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='1000', repl='',regex=False)  # 1000 대신 공백으로 대체. 코인 티커만 남김.

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'binance_future'
    result_df['buyable_price'] = orderbook_list['askPrice']
    result_df['sellable_price'] = orderbook_list['bidPrice']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def ftx_future_price_get(vol_condition = 50000):    # 0.05~0.1초 소요
    url = "https://ftx.com/api/markets"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"result":')[1][:-2])
    orderbook_list = orderbook_list[orderbook_list['name'].str.endswith('PERP')]  # USDT로 끝나는 종목명만 남김
    orderbook_list = orderbook_list[orderbook_list['volumeUsd24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집
    orderbook_list['symbol'] = orderbook_list['name'].str.replace(pat='-PERP', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol'].str.upper()
    result_df['exchange'] = 'FTX_future'
    result_df['buyable_price'] = orderbook_list['ask']
    result_df['sellable_price'] = orderbook_list['bid']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def gate_io_future_price_get(vol_condition = 50000):          # 0.3~0.4초 소요, 종가만 됨.
    url = "https://api.gateio.ws/api/v4/futures/usdt/tickers"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text)
    orderbook_list = orderbook_list[orderbook_list['contract'].str.endswith('USDT')]  # USD나 USDT로 끝나는 종목만 모음
    orderbook_list['contract'] = orderbook_list['contract'].str.replace(pat='_USDT', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['volume_24h_quote'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['contract']
    result_df['exchange'] = 'gate.io_future'
    result_df['buyable_price'] = orderbook_list['last'].astype('float')
    result_df['sellable_price'] = orderbook_list['last'].astype('float')
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def kucoin_future_price_get(vol_condition = 50000):           # 종가만 됨
    url = "https://api-futures.kucoin.com/api/v1/contracts/active"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"data":')[1][:-1])
    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDTM')]  # USDT로 끝나는 종목만 찾음
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='USDTM', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['turnoverOf24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'Kucoin_future'
    result_df['buyable_price'] = orderbook_list['lastTradePrice']
    result_df['sellable_price'] = orderbook_list['lastTradePrice']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def huobi_global_future_price_get():          # 후오비 글로벌 데이터 받기. 약 0.25초 소요
    url = "https://api.hbdm.com/linear-swap-ex/market/bbo"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    orderbook_list = pd.read_json(bsobj.text.split('"ticks":')[1].split('],"ts"')[0]+']')
    orderbook_list['symbol'] = orderbook_list['contract_code'].str.replace(pat='-USDT', repl='',regex=False)  # USDT_ 대신 공백으로 대체. 코인 티커만 남김.

    result_df = pd.DataFrame([],columns=['ticker','exchange','buyable_price','sellable_price'])
    result_df['ticker'] = orderbook_list['symbol'].str.upper()
    result_df['exchange'] = 'huobi_global_future'
    for i in range(len(orderbook_list)):
        if isinstance(orderbook_list['ask'][i],list):
            result_df['buyable_price'][i] = float(orderbook_list['ask'][i][0])
            result_df['sellable_price'][i] = float(orderbook_list['bid'][i][0])
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def mexc_future_price_get(vol_condition = 50000):
    url = "https://contract.mexc.com/api/v1/contract/ticker"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"data":')[1][:-1])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='_USDT', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['amount24'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'mexc_future'
    result_df['buyable_price'] = orderbook_list['ask1']
    result_df['sellable_price'] = orderbook_list['bid1']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def mexc_price_get(vol_condition = 50000):
    url = "https://www.mexc.com/open/api/v2/market/ticker"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"data":')[1][:-1])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list = orderbook_list[
        orderbook_list['symbol'].str.contains('3L|3S|4L|4S|5L|5S') == False]  # 해당 행에 문자열이 포함돼있으면 true 아니면 false 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='_USDT', repl='',
                                                                    regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['amount'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'mexc'
    result_df['buyable_price'] = orderbook_list['ask']
    result_df['sellable_price'] = orderbook_list['bid']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def okx_price_get(vol_condition = 50000):
    url = "https://www.okx.com/api/v5/market/tickers?instType=SPOT"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"data":')[1][:-1])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['instId'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['instId'].str.replace(pat='-USDT', repl='',
                                                                    regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['volCcy24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'OKX'
    result_df['buyable_price'] = orderbook_list['askPx']
    result_df['sellable_price'] = orderbook_list['bidPx']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def okx_future_price_get(vol_condition = 50000):
    url = "https://www.okx.com/api/v5/market/tickers?instType=SWAP"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"data":')[1][:-1])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['instId'].str.endswith(('USDT-SWAP','USD-SWAP'))]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['instId'].str.replace(pat='-USDT-SWAP', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='-USD-SWAP', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['vol24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'OKX_future'
    result_df['buyable_price'] = orderbook_list['askPx']
    result_df['sellable_price'] = orderbook_list['bidPx']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def bybit_price_get(vol_condition = 50000):
    url = "https://api.bybit.com/spot/quote/v1/ticker/24hr"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"result":')[1].split(',"ext_info"')[0])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='USDT', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['quoteVolume'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'bybit'
    result_df['buyable_price'] = orderbook_list['bestAskPrice']
    result_df['sellable_price'] = orderbook_list['bestBidPrice']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def bybit_future_price_get(vol_condition = 50000):
    url = "https://api.bybit.com/v2/public/tickers"
    html = requests.get(url)
    bsobj = bs4.BeautifulSoup(html.content, "html.parser")
    bsobj_json = pd.read_json(bsobj.text.split('"result":')[1].split(',"ext_code"')[0])
    orderbook_list = bsobj_json

    orderbook_list = orderbook_list[orderbook_list['symbol'].str.endswith('USDT')]  # USDT로 끝나는 행에 True 아니면 False 반환
    orderbook_list['symbol'] = orderbook_list['symbol'].str.replace(pat='USDT', repl='',regex=False)  # USDT 대신 공백으로 대체. 코인 티커만 남김.
    orderbook_list = orderbook_list[orderbook_list['turnover_24h'] > vol_condition]  # 일 거래량 5만 달러 이상만 수집

    result_df = pd.DataFrame([])
    result_df['ticker'] = orderbook_list['symbol']
    result_df['exchange'] = 'bybit'
    result_df['buyable_price'] = orderbook_list['ask_price']
    result_df['sellable_price'] = orderbook_list['bid_price']
    result_df = result_df.reset_index(drop=True, inplace=False)  # 인덱스 초기화
    result_df = result_df[['exchange', 'ticker', 'buyable_price', 'sellable_price']]  # 컬럼 순서 변경

    return result_df


def comparing_price(data=[],premium_percent=5,mode='future'):
    price_df_final = pd.concat(data)
    price_df_final = price_df_final.sort_values(by='ticker', ascending=True).reset_index(drop=True, inplace=False)
    if mode=='future':
        presents_index = price_df_final[price_df_final['exchange'].str.contains('future') == False].index
        price_df_final.loc[presents_index, 'sellable_price'] = 0

    previous_ticker = ''
    same_coin_list = pd.DataFrame([])
    result_df = []
    for i in range(len(price_df_final)):
        this_df = price_df_final.iloc[[i]]
        this_ticker = price_df_final['ticker'][i]

        if previous_ticker == this_ticker:  # 이전 행과 티커가 같다면 비교 대상에 추가, 아니면 리스트 비움
            same_coin_list = pd.concat([same_coin_list, this_df])
        else:
            if len(same_coin_list) <= 1:
                pass
            else:
                temp_ticker = same_coin_list['ticker'].iloc[0]
                temp_buy_price = same_coin_list['buyable_price'].min()
                temp_sell_price = same_coin_list['sellable_price'].max()
                if temp_sell_price / temp_buy_price > 1+premium_percent/100:
                    exchange_to_buy = same_coin_list[same_coin_list['buyable_price'] == temp_buy_price]['exchange'].iloc[0]
                    exchange_to_sell = same_coin_list[same_coin_list['sellable_price'] == temp_sell_price]['exchange'].iloc[0]
                    profitable_percent = round(100*(temp_sell_price/temp_buy_price-1),2)
                    result_df.append([temp_ticker, temp_buy_price, temp_sell_price, profitable_percent, exchange_to_buy, exchange_to_sell])
            same_coin_list = this_df

        previous_ticker = this_ticker

    result_df = pd.DataFrame(result_df, columns=['코인이름', '살_가격', '팔_가격', '차이_퍼센트', '살_거래소','팔_거래소'])
    return result_df


# 처음부터 전부 실행, 데이터도 알아서 넣음 됨.
def all_execution(vol_cond = 1000000, future_premium = 1, present_premium = 5):
    future_data = [binance_future_price_get(), gate_io_future_price_get(vol_cond), kucoin_future_price_get(vol_cond),huobi_global_future_price_get(),mexc_future_price_get(vol_cond),okx_future_price_get(vol_cond),bybit_future_price_get(vol_cond)]
    presents_data = [binance_price_get(), upbit_price_get(tether),kucoin_price_get(vol_cond), gate_io_price_get(vol_cond), huobi_global_price_get(vol_cond),mexc_price_get(vol_cond),okx_price_get(vol_cond),bybit_price_get(vol_cond)]
    # poloniex_price_get(vol_cond),kraken_price_get(vol_cond),ftx_price_get(vol_cond)
    # , ftx_future_price_get(vol_cond)
    result_futures = comparing_price(future_data+presents_data, premium_percent=future_premium)
    result_presents = comparing_price(presents_data,premium_percent=present_premium,mode='presents')
    result = pd.concat([result_presents,result_futures])
    return result


# 텔레그램으로 실행
token = ''
id =

bot = telegram.Bot(token)
# bot.sendMessage(chat_id=id, text="봇 시작.")
# print('봇 시작')
pd.set_option('display.max_columns',None)
before_gap_df = pd.DataFrame([], columns=['코인이름', '살_가격', '팔_가격', '차이_퍼센트', '살_거래소', '팔_거래소'])
before_new_gap_df = pd.DataFrame([], columns=['코인이름', '살_가격', '팔_가격', '차이_퍼센트', '살_거래소', '팔_거래소'])

#오류 시 테스트
#tether = tether_price()
#premium_df = all_execution(vol_cond = 1000000, present_premium = 6, future_premium = 3)

while 1:
    sta_time=time.time()
    tether = tether_price()
    try:
        premium_df = all_execution(vol_cond = 1000000, present_premium = 6, future_premium = 3)
    except Exception as e:
        print('오류발생.',datetime.datetime.now())
        print(e)
    else:
        not_coin_list = ['DC','KAVA','SRM','TON']

        df = pd.concat([premium_df, before_gap_df])  # 이전 df와 비교하는 알고리즘 시작
        df = df.reset_index(drop=True)  # 인덱스 초기화
        col = list(df.columns)[0]
        df_grp = df.groupby(col)  # 전체 열 비교
        df_di = df_grp.groups  # 딕셔너리로 만들기
        idx = [x[0] for x in df_di.values() if len(x) == 1]  # 인덱스 검토
        new_gap_df = df.loc[idx, :]  # Same as df.reindex(idx)

        isin_con = new_gap_df['코인이름'].isin(not_coin_list)
        new_gap_df = new_gap_df.loc[isin_con == False]

        if len(new_gap_df) != 0 and list(before_new_gap_df['코인이름']) != list(new_gap_df['코인이름']):
            print('갭발생!!!!!\n' + str(new_gap_df) + '\n\n원본 df')
            bot.send_message(chat_id=id,
                             text=str(new_gap_df) + '\n\n' + str(round(time.time() - sta_time, 3)) + ' 초 전 데이터\n' +
                                  '꼭 네트워크 확인하고 보내셈. 오입금 조심.\n\n'+'테더: '+str(round(tether,2))+'원')
            before_new_gap_df = new_gap_df
        else:
            print('\n새 따리 감지 못 함!\n')
        print(premium_df)
        print('테더 가격:',tether)
        print(datetime.datetime.now())

        before_gap_df = premium_df

    time.sleep(60)
